In [4]:
# Let's do our usual data science imports
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import importlib
import seaborn as sns
import os
import subprocess
sns.set(style="darkgrid")

# Import constants
from config import *

# Import Models
from Evaluation import Evaluation
from Simulator import Simulator
from Setup import Setup

In [5]:
# Helper functions
def makeDirStructure(path):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

## Setting up our Setups

In [6]:
# Constructing the default evaluation objects (no need to touch this!)
Katsuura_eval = Evaluation(
    EVALUATION_KATSUURA,
    {
        PARAMETER_POPSIZE: 100,
        PARAMETER_MUTATION: 0.5,
        PARAMETER_SIGMA: 0.1,
        PARAMETER_MOMENTUM: 0
    }
)

Schaffers_eval = Evaluation(
    EVALUATION_SCHAFFERS,
    {
        PARAMETER_POPSIZE: 100,
        PARAMETER_MUTATION: 0.6,
        PARAMETER_SIGMA: 1,
        PARAMETER_MOMENTUM: 0
    }
)

Sphere_eval = Evaluation(
    EVALUATION_SPHERE,
    {
        PARAMETER_POPSIZE: 100,
        PARAMETER_MUTATION: 0.25,
        PARAMETER_SIGMA: 1,
        PARAMETER_MOMENTUM: 0
    }
)

BCF_eval = Evaluation(
    EVALUATION_BCF,
    {
        PARAMETER_POPSIZE: 100,
        PARAMETER_MUTATION: 0.25,
        PARAMETER_SIGMA: 1,
        PARAMETER_MOMENTUM: 0
    }
)

In [10]:
# Initialize a number of setups

# Popsize
katsuura_popsize_setup = Setup(
    Katsuura_eval,
    PARAMETER_POPSIZE, 
    np.linspace(50, 200, 20)
)

schaffers_popsize_setup = Setup(
    Schaffers_eval,
    PARAMETER_POPSIZE,
    np.linspace(50, 200, 20)
)

sphere_popsize_setup = Setup(
    Sphere_eval,
    PARAMETER_POPSIZE,
    np.linspace(10, 200, 20)
)

bcf_popsize_setup = Setup(
    BCF_eval,
    PARAMETER_POPSIZE,
    np.linspace(10, 200, 20)
)

# Momentum
schaffers_momentum_setup = Setup(
    Schaffers_eval,
    PARAMETER_MOMENTUM,
    [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]
)

sphere_momentum_setup = Setup(
    Sphere_eval,
    PARAMETER_MOMENTUM,
    [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]
)

katsuura_momentum_setup = Setup(
    Katsuura_eval,
    PARAMETER_MOMENTUM,
    [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]
)

bcf_momentum_setup = Setup(
    BCF_eval,
    PARAMETER_MOMENTUM,
    [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]
)

## Simulation Code

In [11]:
# Definining the setups
momentum_setups = [bcf_momentum_setup, katsuura_momentum_setup, sphere_momentum_setup, schaffers_momentum_setup]
popsize_setups = [bcf_popsize_setup, katsuura_popsize_setup, sphere_popsize_setup, bcf_popsize_setup]
mainSimulator = Simulator(momentum_setups, 3, "all_momentum_")

In [40]:
# Generating plots for the cells
for setup in momentum_setups:
    mainSimulator.generatePlotForSetup(setup, end_index=100000)

## Generating New Data from EA

In [12]:
# Only run this if you want to make new data
mainSimulator.run()

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


## Simulating instances

In [29]:
# Reading data (optional)
katsuura_data = mainSimulator.readDataForSetup(momentum_setups[1])

In [39]:
mainSimulator.generatePlotForSetup(momentum_setups[1], output_data=katsuura_data)

In [37]:
bcf_data = mainSimulator.readDataForSetup(momentum_setups[0])